In [2]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# 데이터 로드 및 전처리
data = pd.read_excel('varied_situation_texts.xlsx')
okt = Okt()
korean_stopwords = set(['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다'])

def preprocess_text(text):
    # 특수 문자 및 불필요한 문자 제거
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9\s]', '', text)
    
    # 형태소 분석
    tokens = okt.pos(text, norm=True, stem=True)  # 품사 태깅 (POS tagging)
    
    # 명사, 고유명사 및 중요한 단어만 남김
    tokens = [word for word, tag in tokens if tag in ['Noun', 'ProperNoun', 'Adjective']]
    
    # 추가적인 불용어 제거
    tokens = [word for word in tokens if word not in korean_stopwords]
    
    return ' '.join(tokens)

data['processed_text'] = data['column_containing_text'].apply(preprocess_text)

# 자동 라벨링
disaster_keywords = ['홍수', '지진', '태풍', '화재', '재난', '사고', '비', '우박', '바람', 
                     '폭우', '코로나', '연기', '불', '화재', '화상', '방화', '경보', '신고', 
                     '소방차', '소방관', '폭발', '미세먼지', '폭염', '한파', '번개','낙뢰',
                     '눈', '폭설', '눈사태', '추락', '붕괴', '재난', '응급', '긴급', '주의',
                     '구조', '구급차', '사이렌', '교통사고', '위험', '비상', '위기', '테러',
                     '폭탄', '인질', '전쟁', '대피', '감염', '바이러스', '격리', '정전', 
                     '해킹', '유출', '안전', '생명']

def label_disaster(text):
    for keyword in disaster_keywords:
        if keyword in text:
            return 1
    return 0

data['disaster_column'] = data['processed_text'].apply(label_disaster)

# TF-IDF 피처 추출
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['processed_text'])
y = data['disaster_column']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 나이브 베이즈 모델 학습 및 평가
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== Naive Bayes ===")
print(classification_report(y_test, y_pred, zero_division=1))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\n")

# 전체 데이터에 대해 예측 및 결과 저장
data['predicted_Naive_Bayes'] = model.predict(X)

# 예측 결과 엑셀 파일 저장
data.to_excel('final_disaster_predictions.xlsx', index=False)

=== Naive Bayes ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       124
           1       1.00      1.00      1.00        77

    accuracy                           1.00       201
   macro avg       1.00      1.00      1.00       201
weighted avg       1.00      1.00      1.00       201

Accuracy: 1.0


